In [39]:
import pandas as pd 
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("../testing_ground/final_dataset1.csv")
# df = df.sort_values('user_id',axis=0)
df.head(10)
# df['name'].nunique()

,user_id,name,rating,appid
0,151603712,the elder scrolls v skyrim,5,72850
1,59945701,the elder scrolls v skyrim,3,72850
2,92107940,the elder scrolls v skyrim,5,72850
3,250006052,the elder scrolls v skyrim,5,72850
4,11373749,the elder scrolls v skyrim,5,72850
5,54103616,the elder scrolls v skyrim,2,72850
6,56038151,the elder scrolls v skyrim,1,72850
7,94088853,the elder scrolls v skyrim,5,72850
8,116085629,the elder scrolls v skyrim,4,72850
9,148510973,the elder scrolls v skyrim,4,72850


In [49]:
user_ratings = df.pivot_table(index=['user_id'],columns=['name'],values='rating')
user_ratings = user_ratings.dropna(thresh=0,axis=1).fillna(0)
user_ratings

name,7 days to die,adventure capitalist,age of empires ii hd edition,age of empires iii complete collection,age of mythology extended edition,age of wonders iii,alan wake,alien isolation,alien swarm,aliens vs predator,...,warhammer 40000 dawn of war soulstorm,warhammer 40000 dawn of war ii,warhammer 40000 dawn of war ii chaos rising,warhammer 40000 space marine,watchdogs,wolfenstein the new order,worms reloaded,worms revolution,xcom enemy unknown,zombie panic source
user_id,,,,,,,,,,,,,,,,,,,,,
5250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76767,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
229911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
298950,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309255941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309265377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309404240,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
## Building the similarity matrix
game_similarity_df = user_ratings.corr(method='pearson',min_periods=0)
game_similarity_df

name,7 days to die,adventure capitalist,age of empires ii hd edition,age of empires iii complete collection,age of mythology extended edition,age of wonders iii,alan wake,alien isolation,alien swarm,aliens vs predator,...,warhammer 40000 dawn of war soulstorm,warhammer 40000 dawn of war ii,warhammer 40000 dawn of war ii chaos rising,warhammer 40000 space marine,watchdogs,wolfenstein the new order,worms reloaded,worms revolution,xcom enemy unknown,zombie panic source
name,,,,,,,,,,,,,,,,,,,,,
7 days to die,1.000000,0.040443,0.064832,0.015247,-0.004595,0.049615,0.020661,0.026756,0.030640,0.032369,...,0.012734,0.046931,0.072669,-0.001752,0.073850,0.033908,0.070160,0.008993,0.027882,0.001559
adventure capitalist,0.040443,1.000000,0.049808,0.021506,-0.004754,0.013783,0.003897,-0.002207,0.013149,-0.006790,...,-0.004369,-0.008922,-0.004294,-0.002174,-0.005317,-0.003299,-0.004771,-0.002466,0.010281,-0.003963
age of empires ii hd edition,0.064832,0.049808,1.000000,0.145065,0.138550,0.050145,0.060622,0.022707,0.066156,0.008574,...,0.013191,0.062207,0.069323,0.078859,0.030677,0.040724,0.052159,0.061695,0.035236,0.008268
age of empires iii complete collection,0.015247,0.021506,0.145065,1.000000,0.098277,0.043145,0.005926,-0.003279,0.020822,0.001916,...,0.017796,0.041063,0.076412,0.095901,-0.000670,0.008472,0.058987,-0.000741,0.036629,0.027272
age of mythology extended edition,-0.004595,-0.004754,0.138550,0.098277,1.000000,0.100701,0.036987,0.006346,0.006112,0.033140,...,0.070314,0.055807,0.024575,0.019206,0.030785,0.013525,-0.002860,0.076032,0.040963,0.028528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wolfenstein the new order,0.033908,-0.003299,0.040724,0.008472,0.013525,0.111708,0.090171,0.135775,0.003429,0.068596,...,0.032890,0.010746,0.009331,0.083270,0.052254,1.000000,0.057392,0.007694,0.119356,-0.003356
worms reloaded,0.070160,-0.004771,0.052159,0.058987,-0.002860,0.052073,0.059050,0.038026,0.057897,0.057089,...,0.068999,0.041396,0.016344,0.062888,0.116912,0.057392,1.000000,0.027978,0.021312,0.005811
worms revolution,0.008993,-0.002466,0.061695,-0.000741,0.076032,0.042037,0.058164,0.035705,0.065601,0.036025,...,0.065490,0.051100,0.098223,0.042096,0.045864,0.007694,0.027978,1.000000,0.133370,0.014181


In [53]:
def get_similar_games(game_name,user_rating):
    similar_score = game_similarity_df[game_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score.head(10)

print(get_similar_games('the witcher 3 wild hunt',5))

name
counterstrike                     2.500000
counterstrike condition zero      0.816377
counterstrike global offensive    0.397065
day of defeat                     0.264301
counterstrike source              0.228468
fear 3                            0.162985
cry of fear                       0.148882
dino dday                         0.145004
payday the heist                  0.143138
grand theft auto iv               0.125427
Name: counterstrike, dtype: float64
